# VNA V3

Want to be able to live plot both S-parameters and the data-read in at once. Also cleans up the imports.

In [1]:
import sys
import os
import importlib #
sys.path.append(os.path.abspath('../scripts'))

#widgets and display
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import Output
import pprint

#Plotting 
from bqplot import pyplot as plt
import threading
from threading import Thread

#caculations
import numpy as np 
from scipy import signal
import time
import math

# Import functions from custom scripts
import sig_source, sig_source_ui, data_proccess
importlib.reload(sig_source)
importlib.reload(sig_source_ui)
importlib.reload(data_proccess)
from sig_source import SigSource
from sig_source_ui import SignalSourceUI
from data_proccess import VNAfunc

In [2]:
#Import Pynq
from pynq import PL
from pynq import allocate
from pynq import Overlay
import xrfdc

### Upload Code to RFSOC

In [3]:
rfsoc_button = widgets.Button(description="Print RFSOC Code Lines")

rfosc_out = Output()
def print_rfsoc(func):
    with rfosc_out:
        rfosc_out.clear_output
        try: 
            pprint.pprint(ol.ip_dict)
        except Exception as e:
            print(f"Error: {e}")


PL.reset() #important fixes caching issues which have popped up.
ol = Overlay('../design_1.bit')  #locate/point to the bit file
dma_interface = ol.axi_dma_0
print(ol.ip_dict.keys())

rf = ol.usp_rf_data_converter_0

rfsoc_button.on_click(print_rfsoc)
display(widgets.VBox([widgets.Label(value="Print RFSOC Output"), rfsoc_button, rfosc_out])) #Display button only after the code is uploaded 

dict_keys(['axi_dma_0', 'spi_adl_1', 'spi_lmx_0', 'spi_adl_0', 'usp_rf_data_converter_0', 'zynq_ultra_ps_e_0'])


### Signal Source Generation 

In [4]:
# Create and display the UI for signal generation
sig_ui = SignalSourceUI(mmio_spi_controller = ol.spi_lmx_0)
sig_ui.display()

Initalized source


In [5]:
#sig_ui.source.set_frequency(10000000)

Fvco: 9600.0
Fden: 1000
PreR: 1
OSC_2x: 1
MULT: 1
MultOut: 200.0
Rdiv: 1
Setting PLL_N to 48


In [10]:
# Global control variable
running = False

# Widgets with layout adjustments
mode_toggle = widgets.ToggleButtons(
    options=['User Input Frequency', 'Generate Points'],
    description='Mode:',
    disabled=False,
    button_style='',
    layout=widgets.Layout(
        width='auto',         # Automatically adjust width based on content
        min_width='400px',    # Minimum width to prevent truncation
        max_width='1000px'    # Max width to avoid excessive stretching
    )
)
user_freq_input = widgets.FloatText(
    description='Freq (Hz):',
    value=10e6,  # Example starting frequency
    step=1,
    layout=widgets.Layout(width='300px')  # Adjust width to fit description
)
set_freq_button = widgets.Button(
    description="Set Frequency",
    layout=widgets.Layout(width='150px')  # Adjust width for clarity
)
generate_freq_button = widgets.Button(
    description="Generate Frequency Points",
    layout=widgets.Layout(width='200px')  # Adjust width to fit text
)
# Output widget
output = widgets.Output(layout=widgets.Layout(width='500px'))  # Adjust width for output clarity

# Functions
def freq_update_loop(frequency_list):
    global running
    for freq in frequency_list:
        if not running:  # Check if the loop should be stopped
            break
        # Simulate frequency update (replace this with the actual signal generation function)
        with output:
            clear_output(wait=True)
            print(f"Setting freq to {freq:.2f}")
            sig_ui.source.set_frequency(freq)
        #TO DO REMOVE TIME HERE
        time.sleep(0.3)  # Simulate delay between frequency updates

def set_user_frequency(change=None):
    with output:
        clear_output()
        try:
            if mode_toggle.value == "User Input Frequency":
                frequency = user_freq_input.value
                sig_ui.source.set_frequency(frequency) #Calls function to set frequency
                print(f"Set frequency to {frequency} Hz.")
            else:
                print(f"Wrong Mode. Following automatic Frequency")
        except Exception as e:
            print(f"Error setting frequency: {e}")

def generate_freq_points(change=None):
    with output:
        clear_output()
        try:
            if mode_toggle.value == "Generate Points":
                frequency_list = sig_ui.source.generate_freq_points()
                # Start the frequency update loop in a new thread
                thread = threading.Thread(target=freq_update_loop, args=(frequency_list,), daemon=True)
                thread.start()
            else:
                print(f"Wrong Mode. Following user input Frequency")
        except Exception as e:
            print(f"Error generating frequency points: {e}")

# Handle mode changes to stop the loop
def on_mode_toggle_change(change):
    global running
    if mode_toggle.value == "User Input Frequency":
        running = False
        with output:
            clear_output(wait=True)
            print("Mode changed to User Input Frequency. Stopping frequency generation.")
    else:
        running = True
        with output:
            clear_output(wait=True)
            print("Mode changed to Generate Frequency Points. Hit Generate Frequency Points to start sweep.")

# Widgets event bindings
set_freq_button.on_click(set_user_frequency)
generate_freq_button.on_click(generate_freq_points)
mode_toggle.observe(on_mode_toggle_change, names='value')

# Layout
layout = widgets.VBox([
    mode_toggle,
    widgets.HBox([user_freq_input, set_freq_button]),
    generate_freq_button,
    output
])
display(layout)


### Real Time Output Plot

In [7]:
fs = 147.456e6 # Sampling frequency
n = 65536 # Number of samples
T = n/fs

In [8]:
#Import Relavent Functions 
def read_dma():
    # Trigger the DMA transfer and wait for the result
    out_buffer = allocate(400024 * 4, dtype=np.int32)
    # Trigger the DMA transfer and wait for the result
    dma_interface.recvchannel.transfer(out_buffer)
    dma_interface.recvchannel.wait()
    return out_buffer

In [9]:
# Plot Size
PLOT_SIZE = 3e-6

# Initialize data lists
time_data = np.linspace(0, T, n)  # Time data (X-axis)
x_axis = [time_data, time_data, time_data, time_data]
# Initialize the plot
fig = plt.figure(title="Real-time Sensor Data", animation_duration=0)
line = plt.plot([], [], colors=["blue", "red", "green", "orange"])  # Initial empty plot

plt.xlim(0, PLOT_SIZE)  # Initial X-axis range, will update dynamically
plt.xlabel("Time [s]")

# Function to update the plot with new sensor data
def update_plot():
    while True:
        if is_running.value:
            out_buffer = read_dma()

            # Convert the entire out_buffer into a NumPy array
            out_buffer_np = np.array(out_buffer, dtype=np.int32)  # Assuming `out_buffer` is the raw input array

            # Split into 4 sub-arrays
            out_buffer0 = out_buffer_np[0::4]
            out_buffer1 = out_buffer_np[1::4]
            out_buffer2 = out_buffer_np[2::4]
            out_buffer3 = out_buffer_np[3::4]

            # Process all buffers at once
            real0, imag0 = VNAfunc.iq_break_data_np(out_buffer0)
            real1, imag1 = VNAfunc.iq_break_data_np(out_buffer1)
            real2, imag2 = VNAfunc.iq_break_data_np(out_buffer2)
            real3, imag3 = VNAfunc.iq_break_data_np(out_buffer3)
                
            #Caculate the S-parameters 
            #For caculating the S-parameters combine into complex lists
            array0 = real0 + 1j*imag0 
            array1 = real1 + 1j*imag1
            array2 = real2 + 1j*imag2 
            array3 = real3 + 1j*imag3 

            S11_mag, S11_phase = VNAfunc.caculate_S_param(array0, array1)
            S12_mag, S12_phase = VNAfunc.caculate_S_param(array0, array2)
            S21_mag, S21_phase = VNAfunc.caculate_S_param(array1, array3)
            S22_mag, S22_phase = VNAfunc.caculate_S_param(array2, array3)

#             print(f"Calculated S11 magnitude is {S11_mag:.2f}.")
#             print(f"Calculated S11 phase is {S11_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}.")
#             print(f"Calculated S12 magnitude is {S12_mag:.2f}.")
#             print(f"Calculated S12 phase is {S12_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}.")
#             print(f"Calculated S21 magnitude is {S21_mag:.2f}.")
#             print(f"Calculated S21 phase is {S21_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}.")
#             print(f"Calculated S22 magnitude is {S22_mag:.2f}.")
#             print(f"Calculated S22 phase is {S22_phase * 180 / math.pi:.2f}\N{DEGREE SIGN}."
            
            # Update the plot with new data
            line.x = x_axis
            line.y = [real0, real1, real2, real3]
            
# Toggle button to start/stop the plot
is_running = widgets.ToggleButton(
    value=False,
    description="Running",
    icon="play",
    tooltip="Start/Stop the live plot",
)

# Display the toggle button and plot
display(is_running, fig)

# Function to start the thread for continuous plotting
def start_plot(change):
    if is_running.value:
        # Run the update function in a separate thread to avoid blocking the main thread
        thread = Thread(target=update_plot, daemon=True)
        thread.start()

# Watch the button and start the plot when pressed
is_running.observe(start_plot, names='value')

ToggleButton(value=False, description='Running', icon='play', tooltip='Start/Stop the live plot')

Figure(axes=[Axis(label='Time [s]', scale=LinearScale(max=3e-06, min=0.0)), Axis(orientation='vertical', scale…